In [ ]:
import numpy as np
from sklearn import datasets
iris = datasets.load_iris()
X_train = iris.data
y_train = iris.target
X_train = np.column_stack((X_train, iris.target))
features = iris.feature_names

In [ ]:
target_names = iris.target_names
label = list(target_names)
label

In [ ]:
class Question:
    def __init__(self, column, value):
        self.column = column
        self.value = value
    
    def match(self, sample):
        return (sample[self.column] >= self.value)
    
    def __repr__(self):
        return 'Is %s %s %s?' % (features[self.column], '>=' , str(self.value))
    

In [ ]:
def partition(rows, question):
    
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
        
    return true_rows, false_rows

In [ ]:
def class_counts(rows):
    
    counts = {}
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    
    return counts

In [ ]:
class_counts(X_train)

In [ ]:
def gini(rows):
    
    counts = class_counts(rows)
    impurity = 1
    for i in counts:
        probability = counts[i] / len(rows)
        impurity = impurity - (probability)**2
    
    return impurity

In [ ]:
gini(X_train)

In [ ]:
def info_gain(left, right, curr_impurity):
    
    l = float(len(left)) / len(left) + len(right)
    return curr_impurity - l*gini(left) - (1-l)*gini(right)


In [ ]:
def best_split(rows):
    features = len(rows[0]) - 1
    curr_impurity = gini(rows)
    best_gain = 0
    best_question = None
    
    for col in range(features):     #for every feature(column)
        value_set = set([row[col] for row in rows])
        for val in value_set:
            question = Question(col, val)
            true_rows, false_rows = partition(rows, question)
            
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
        
            gain = info_gain(true_rows, false_rows, curr_impurity)
            if gain >= best_gain:
                best_gain, best_question = gain, question
            
    return best_gain, best_question
    

In [ ]:
bg, bq = best_split(X_train)

In [ ]:
print(bg)
print(bq)

In [ ]:
class leaf:
    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [ ]:
def build_tree(rows):
    info, question = best_split(rows)
    
    if info == 0:
        return leaf(rows)
    
    true_rows, false_rows = partition(rows, question)
    #recursion
    true_subtree = build_tree(true_rows)
    false_subtree = build_tree(false_rows)
    
    return Decision_Node(question, true_subtree, false_subtree)

In [ ]:
class Decision_Node:
    def __init__(self, question, true_subtree, false_subtree):
        self.question = question
        self.true_subtree = true_subtree
        self.false_subtree = false_subtree

In [ ]:
def print_tree(node, spacing=""):
    if isinstance(node, leaf):   #base case
        print (spacing + "Predict", node.predictions)
        return

    print (spacing + str(node.question))

    print (spacing + '--> True:')
    print_tree(node.true_subtree, spacing + "  ")

    
    print (spacing + '--> False:')
    print_tree(node.false_subtree, spacing + "  ")

In [ ]:
iris_dt = build_tree(X_train)

In [ ]:
print_tree(iris_dt)